In [1]:
import numpy as np
import tensorflow as tf
import pickle
import tqdm
from sklearn.model_selection import train_test_split
import os

tfk = tf.keras
tfkl = tf.keras.layers

tf.random.set_seed(42)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
physical_devices = tf.config.list_physical_devices('GPU')
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, True)

find_max_len = 1
mask_val = -1.

train = 1
save_dir = './models/ludo_qval_rnn_11_to_15_v2'

2022-09-21 16:21:55.183817: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-09-21 16:21:58.767205: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-21 16:21:58.768315: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-21 16:21:59.041590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 16:21:59.045007: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:05.0 name: NVIDIA Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-09-21 16:21:59.045050: I tensorflow/stream_executor/platform/default/ds

In [2]:
with open('../data/ludo_qval_rnn_features_11_to_15.pkl', 'rb') as f:
    feats, y = pickle.load(f)

In [3]:
length = 0
for feat in tqdm.tqdm(feats):
    for i in range(2):
        for j in range(5):
            temp = len(feat['q_user_'+str(i)+'_game_'+str(j)])
            if temp > length:
                length = temp

100%|██████████| 450826/450826 [00:04<00:00, 90838.50it/s]


In [4]:
for i in range(2):
    globals()['user_'+str(i)+'_mu'] = []
    globals()['user_'+str(i)+'_sigma'] = []
    for j in range(5):
        globals()['q_user_'+str(i)+'_game_'+str(j)] = []

for f, feat in enumerate(tqdm.tqdm(feats)):
    for i in range(2):
        for j in range(5):
            temp_len = len(feat['q_user_'+str(i)+'_game_'+str(j)])
            if temp_len < length:
                temp1 = np.expand_dims(np.concatenate((feat['q_user_'+str(i)+'_game_'+str(j)], mask_val*np.ones(length-temp_len)), 0), -1)
                temp2 = np.expand_dims(np.concatenate((feat['q_user_'+str(i)+'_max_game_'+str(j)], mask_val*np.ones(length-temp_len)), 0), -1)
            # elif temp_len == 1:
            #     temp1 = mask_val*np.ones((length,1))
            #     temp2 = mask_val*np.ones((length,1))
            else:
                temp1 = np.expand_dims(feat['q_user_'+str(i)+'_game_'+str(j)], -1)
                temp2 = np.expand_dims(feat['q_user_'+str(i)+'_max_game_'+str(j)], -1)
            globals()['q_user_'+str(i)+'_game_'+str(j)].append(np.concatenate((temp1, temp2), -1))
        globals()['user_'+str(i)+'_mu'].append(feat['user_'+str(i)+'_mu'])
        globals()['user_'+str(i)+'_sigma'].append(feat['user_'+str(i)+'_sigma'])

for i in range(2):
    globals()['user_'+str(i)+'_mu'] = np.array(globals()['user_'+str(i)+'_mu'])
    globals()['user_'+str(i)+'_sigma'] = np.array(globals()['user_'+str(i)+'_sigma'])
    for j in range(5):
        globals()['q_user_'+str(i)+'_game_'+str(j)] = np.array(globals()['q_user_'+str(i)+'_game_'+str(j)])

skill_feats = np.concatenate((np.expand_dims(user_0_mu,-1), 
                              np.expand_dims(user_1_mu,-1), 
                              np.expand_dims(user_0_sigma,-1), 
                              np.expand_dims(user_1_sigma,-1)), -1)

100%|██████████| 450826/450826 [02:23<00:00, 3150.02it/s]


In [5]:
for i in range(2):
    for j in range(5):
        globals()['q_user_'+str(i)+'_game_'+str(j)+'_train'], globals()['q_user_'+str(i)+'_game_'+str(j)+'_test'] = train_test_split(
            globals()['q_user_'+str(i)+'_game_'+str(j)], test_size=0.25, random_state=42)
skill_feats_train, skill_feats_test, y_train, y_test = train_test_split(skill_feats, y, test_size=0.25, random_state=42)

In [6]:
for i in range(2):
    for j in range(5):
        globals()['q_user_'+str(i)+'_game_'+str(j)+'_train'], globals()['q_user_'+str(i)+'_game_'+str(j)+'_val'] = train_test_split(
            globals()['q_user_'+str(i)+'_game_'+str(j)+'_train'], test_size=0.2, random_state=42)
skill_feats_train, skill_feats_val, y_train, y_val = train_test_split(skill_feats_train, y_train, test_size=0.2, random_state=42)

In [7]:
def rnn_model(units):
    skill = tfkl.Input(shape=(6, 4))
    skill_feats = tfkl.LSTM(units, return_sequences=True)(skill)
    skill_feats = tfkl.LSTM(units, return_sequences=True)(skill_feats)
    skill_feats = tfkl.LSTM(units, return_sequences=True)(skill_feats)
    skill_feats = tfkl.LSTM(units, return_sequences=False)(skill_feats)

    x = []
    inps = []
    
    x.append(skill_feats)
    inps.append(skill)

    for i in range (10):
        locals()['inp_'+str(i)] = tfkl.Input(shape=(length, 2))
        locals()['x_'+str(i)] = tfkl.Masking(mask_value=mask_val)(locals()['inp_'+str(i)])
        locals()['x_'+str(i)] = tfkl.LSTM(units, return_sequences=True)(locals()['x_'+str(i)])
        locals()['x_'+str(i)] = tfkl.LSTM(units, return_sequences=True)(locals()['x_'+str(i)])
        locals()['x_'+str(i)] = tfkl.LSTM(units, return_sequences=False)(locals()['x_'+str(i)])
        x.append(locals()['x_'+str(i)])
        inps.append(locals()['inp_'+str(i)])

    x = tfkl.Concatenate()(x)
    x = tfkl.Dense(256, activation='elu')(x)
    x = tfkl.Dense(128, activation='elu')(x)
    x = tfkl.Dense(64, activation='elu')(x)
    x = tfkl.Dense(32, activation='elu')(x)
    out = tfkl.Dense(2, activation='softmax')(x)

    return tfk.Model(inputs=inps, outputs=out)

In [8]:
train_set = []
val_set = []
test_set = []

train_set.append(skill_feats_train)
val_set.append(skill_feats_val)
test_set.append(skill_feats_test)

for i in range (2):
    for j in range (5):
        train_set.append(globals()['q_user_'+str(i)+'_game_'+str(j)+'_train'])
        val_set.append(globals()['q_user_'+str(i)+'_game_'+str(j)+'_val'])
        test_set.append(globals()['q_user_'+str(i)+'_game_'+str(j)+'_test'])

In [9]:
model = rnn_model(20)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics='accuracy')

if train:

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=save_dir,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

    model.fit(train_set, y_train, batch_size=512, epochs=100, validation_data=(val_set, y_val), callbacks=[model_checkpoint_callback])

model.load_weights(save_dir)

2022-09-21 16:25:21.423328: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-21 16:25:21.425804: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-21 16:25:21.426064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 16:25:21.427886: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:05.0 name: NVIDIA Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
202

Epoch 1/100


2022-09-21 16:27:15.506163: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-09-21 16:27:16.044222: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


529/529 [==============================] - 223s 248ms/step - loss: 0.6905 - accuracy: 0.5373 - val_loss: 0.6912 - val_accuracy: 0.5399
Epoch 2/100
529/529 [==============================] - 97s 183ms/step - loss: 0.6896 - accuracy: 0.5387 - val_loss: 0.6893 - val_accuracy: 0.5402
Epoch 3/100
529/529 [==============================] - 96s 182ms/step - loss: 0.6888 - accuracy: 0.5404 - val_loss: 0.6893 - val_accuracy: 0.5403
Epoch 4/100
529/529 [==============================] - 96s 182ms/step - loss: 0.6888 - accuracy: 0.5404 - val_loss: 0.6885 - val_accuracy: 0.5409
Epoch 5/100
529/529 [==============================] - 96s 182ms/step - loss: 0.6880 - accuracy: 0.5426 - val_loss: 0.6881 - val_accuracy: 0.5443
Epoch 6/100
529/529 [==============================] - 97s 182ms/step - loss: 0.6879 - accuracy: 0.5426 - val_loss: 0.6879 - val_accuracy: 0.5444
Epoch 7/100
529/529 [==============================] - 97s 182ms/step - loss: 0.6876 - accuracy: 0.5438 - val_loss: 0.6880 - val_accura

In [10]:
y_probs = model.predict(test_set)
y_pred = np.argmax(y_probs,-1)

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

def perf(ytrue, ypred):
    accu = accuracy_score(ytrue, ypred)
    prec = precision_score(ytrue, ypred)
    rec = recall_score(ytrue, ypred)
    return accu*100, prec*100, rec*100

In [12]:
acc, precision, recall = perf(y_test, y_pred)
print(acc, precision, recall)

54.26104855953934 54.69318457433907 88.95609724025707


In [13]:
bins = np.abs(y_probs[:,1]-0.5)

In [14]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*0.05, bins<(i+1)*0.05)
    temp = y_test[idx]
    u1.append(np.sum(temp[y_probs[:,1][idx]>=0.5]==1) + np.sum(temp[y_probs[:,1][idx]<0.5]==0))
    tot.append(len(y_test[idx]))

In [15]:
import pandas as pd
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,28498,54777
1,24701,44318
2,7255,12399
3,683,1179
4,18,31
5,1,3
6,0,0
7,0,0
8,0,0
9,0,0
